In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
import requests
from datetime import datetime

In [2]:
r = requests.get("https://www.tert.am/am/news/politics/1")
sel = scrapy.Selector(text=r.text)

In [3]:
sel.css("span.list__title::text").extract()

['Իրանը տարածաշրջանի կայունության և անվտանգության խարիսխն է, ոչ թե «թավշյա» կամ «գունավոր» հեղաշրջումների երկիր. Ամիր Աբդոլահիան',
 'Արցախի ինքնորոշման իրավունքի նարատիվը ամեն գնով պետք է վերականգնել, ու քանի որ Հայաստանի իշխանությունները հրաժարվում են դրանից, դա կարող են անել հենց արցախցիները. Վարդան Օսկանյան',
 'Արցախի ցեղասպանությունը կլինի նաև ժողովրդավարական աշխարհի քաղաքականության արդյունք․ Դավիթ Բաբայան',
 'Ալիևն ինչ ուզում է տալիս են, Երևանին էլ հերթը կհասնի, եթե այս իշխանությունները մնան․ Տիգրան Թորոսյան (տեսանյութ)',
 'Պարգևավճարից չեմ հրաժարվում, օգտվում եմ՝ օգտագործելու տեղեր կան, իսկ ընդդիմությունը լավ չի աշխատում, ճիշտ է պարգևավճար չտալը․ Հովիկ Աղազարյան․ 24news.am',
 'Թուրքիայի և Ադրբեջանի ներկայացուցիչները ընդհատել են ԵԽԽՎ-ում Հայկ Մամիջանյանի ելույթը, սակայն ՔՊ-ական պատգամավորների միջամտությունից հետո ելույթը շարունակվել է',
 'ՊՆ-ն հերքում է ականի պայթյունից հայ զինծառայողների զոհվելու և վիրավորելու մասին ադրբեջանական տեղեկատվությունը',
 'Հատուկ ներկայացուցիչների հանդի

In [4]:
sel.css("span.list__date::text").extract()

['21:06 • 15.10.22',
 '19:50 • 15.10.22',
 '23:01 • 14.10.22',
 '21:04 • 14.10.22',
 '20:20 • 14.10.22',
 '18:00 • 14.10.22',
 '17:34 • 14.10.22',
 '16:21 • 14.10.22',
 '16:20 • 14.10.22',
 '13:00 • 14.10.22',
 '12:05 • 14.10.22',
 '23:27 • 13.10.22',
 '22:44 • 13.10.22',
 '21:35 • 13.10.22',
 '15:55 • 13.10.22']

In [5]:
sel.css("a.list__link::attr(href)").extract()

['/am/news/2022/10/15/iran/3851052',
 '/am/news/2022/10/15/oskanyan/3851034',
 '/am/news/2022/10/14/Davit-babayan/3850783',
 '/am/news/2022/10/14/tigran-torosyan/3850542',
 '/am/news/2022/10/14/aghazaryan/3850709',
 '/am/news/2022/10/14/mamijanyan/3850682',
 '/am/news/2022/10/14/pn-akan/3850672',
 '/am/news/2022/10/14/agn-mfa/3850634',
 '/am/news/2022/10/14/bayramov/3850632',
 '/am/news/2022/10/14/petakan-pahpanutyan-tsarayutyun/3850512',
 '/am/news/2022/10/14/putin-macron/3850495',
 '/am/news/2022/10/13/pace-russia/3850341',
 '/am/news/2022/10/13/artur-tovmasyan/3850330',
 '/am/news/2022/10/13/pace-armenia/3850301',
 '/am/news/2022/10/13/zurabyan/3850135']

In [ ]:
titles = []
dates = []
contents = []

class TertCrawler(scrapy.Spider):
    name = "MovsisyansNewsCrawler"

    custom_settings=dict(
        DOWNLOAD_DELAY = 0.08
    )
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.domain = "https://www.tert.am" 
        self.scopes = [
            # "/am/news/politics/",
            # "/am/news/Կորոնավիրուս/",
            # "/am/news/law/",
            # "/am/news/business/",
            # "/am/news/Sports/",
            #"/am/news/event/",
            # "/am/news/press-digest/",
            # "/am/news/culture/",
            "/life/am/news/fashion/",
            "/life/am/news/interview/",
            "/life/am/news/showbiz/",
            "/life/am/news/entertainment/",
            "/life/am/news/photo/",
            "/life/am/news/tv/",
            "/life/am/news/culture/"
        ]
        # infer pages programmatically
        self.pages = [76, 28, 373, 286, 15, 14, 7]

    def start_requests(self):
        urls = [self.scopes[scope_id] + str(page) 
                for scope_id in range(len(self.scopes)) 
                for page in range(1, self.pages[scope_id])]
        
        for url in urls:
            yield scrapy.Request(url=self.domain + url, callback=self.parse_page)

    def parse_page(self, response):
        links = response.css("a.list__link::attr(href)").extract()

        for link in links:
            yield response.follow(url=link, callback=self.parse_article)



    def parse_article(self, response):
        title = response.css("h1.inner-content__article-title::text").extract_first()
        dt = response.css("div.inner-content__article-date::text").extract_first()
        date = datetime.strptime(dt, "%H:%M • %d.%m.%y")
        content = " ".join([i.strip().replace("\xa0", " ") for i in response.css("div#news-content-container p::text").extract()])
        # add link capture

        # add better data streaming
        titles.append(title)
        dates.append(date)
        contents.append(content) 